In [1]:
from flair.data import Sentence
from flair.nn import Classifier
from flair.models import SequenceTagger
#from flair.models import MultiTagger
from flair.models import MultitaskModel
import flair

2023-07-19 12:22:27.183117: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 12:22:28.125292: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
import json
import nltk
import pandas as pd
from tqdm import tqdm
from warnings import filterwarnings
#from articleDetails import ArticleDetails

In [3]:
#tagger = Classifier.load("hunflair")

In [16]:
training_datasets=flair.datasets.biomedical.BIOBERT_CHEMICAL_BC4CHEMD(base_path="/home/aberhe/Projects/SANTAL/ASLR/data/BC4CHEMD")

Unzipping...

/home/aberhe/.local/lib/python3.10/site-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1OletxmPYNkz2ltOr9pyT0b0iBtUWxslh" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


FileNotFoundError: [Errno 2] No such file or directory: '/home/aberhe/Projects/SANTAL/ASLR/data/BC4CHEMD/biobert_common/BC4CHEMD/train.tsv'

In [10]:
training_datasets

NameError: name 'training_datasets' is not defined

In [22]:
len(training_datasets.train.fget())

TypeError: Corpus.train() missing 1 required positional argument: 'self'

In [3]:
sentence = Sentence("Behavioral abnormalities in the Fmr1 KO2 Mouse Model of Fragile X Syndrome")
tagger = Classifier.load("hunflair")
df=pd.DataFrame(columns=("sentence","entity","start_chunck","end_chunk","label","score"))

#sentence=nltk.sent_tokenize(document)
tagger.predict(sentence)


2023-07-19 12:22:51,699 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, B-Disease, E-Disease, I-Disease, S-Disease, <START>, <STOP>
2023-07-19 12:22:52,631 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, S-Gene, B-Gene, I-Gene, E-Gene, <START>, <STOP>
2023-07-19 12:22:53,558 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, S-Species, B-Species, I-Species, E-Species, <START>, <STOP>
2023-07-19 12:22:54,467 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, S-CellLine, B-CellLine, I-CellLine, E-CellLine, <START>, <STOP>
2023-07-19 12:22:55,387 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, S-Chemical, B-Chemical, I-Chemical, E-Chemical, <START>, <STOP>


In [4]:
tagger.to('cuda')

MultitaskModel(
  (Task_0): SequenceTagger(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        '/home/alan/.flair/embeddings/pubmed_pmc_wiki_sg_1M.gensim'
        (embedding): Embedding(1000001, 200)
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.1, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=275, bias=True)
        )
      )
    )
    (word_dropout): WordDropout(p=0.05)
    (locked_dropout): LockedDropout(p=0.5)
    (embedding2nn): Linear(in_features=4296, out_features=4296, bias=True)
    (rnn): LSTM(42

In [12]:
import multiprocessing as mp
import multiprocessing.spawn.spawn

In [ ]:
mp.sp

In [5]:
df=pd.DataFrame(columns=("sentence","entity","start_chunck","end_chunk","label","score"))

In [4]:
#sentence.get_spans("species")

In [11]:
file_xml='/data/projects/santal/data/PMC_Articles/oa_comm/xml/all/PMC10000167.xml'#PMC10000014.xml'
with open(file_xml, 'r') as f:
    data = f.read()
articleDetails=ad.ArticleDetails(file_xml)
articleDetails.getDetails()
sent_id_d=file_xml.split("/")[-1].replace(".xml","")
df=pd.DataFrame(columns=("sent_id","sentence","entity","start_chunck","end_chunk","label","score"))
i=0
for key in articleDetails.sections:
    sentences = nltk.sent_tokenize(articleDetails.sections[key])
    for sentence in sentences:
        sentence=Sentence(sentence)
        tagger.predict(sentence)
        sent_id=sent_id_d+"_"+str(i)
        for labels in sentence.get_labels():
            #print(la.value,la.data_point.text,la.data_point.start_position,la.data_point.end_position)
            taggedEntities=[sent_id,sentence.tokenized,labels.data_point.text,labels.data_point.start_position,labels.data_point.end_position,labels.value,round(labels.score,3)]
            df.loc[len(df)]=taggedEntities
        i=i+1

FILE READ!!


In [ ]:
df

,sent_id,sentence,entity,start_chunck,end_chunk,label,score
0,PMC10000167_2,Warm - and cool-season grasses have different ...,carbohydrates,79,92,Chemical,0.991
1,PMC10000167_2,Warm - and cool-season grasses have different ...,carbohydrate,118,130,Chemical,0.992
2,PMC10000167_2,Warm - and cool-season grasses have different ...,sugars,138,144,Chemical,0.897
3,PMC10000167_3,These differences in NSC content are of intere...,equine,52,58,Species,0.742
4,PMC10000167_3,These differences in NSC content are of intere...,horses,109,115,Species,0.968
...,...,...,...,...,...,...,...
706,PMC10000167_624,Fecal BCFA and SCFA concentrations were higher...,SCFA,15,19,Chemical,0.930
707,PMC10000167_624,Fecal BCFA and SCFA concentrations were higher...,horses,50,56,Species,0.770
708,PMC10000167_625,The guild-based analytical approach utilized i...,sugar,280,285,Chemical,0.897
709,PMC10000167_627,and Clostridium butyricum in the hindgut micro...,Clostridium butyricum,4,25,Species,0.770


In [23]:
sentTag.get_labels()

[]

In [13]:
sentence.text

'Behavioral abnormalities in the Fmr1 KO2 Mouse Model of Fragile X Syndrome'

In [129]:
sentence = Sentence("Behavioral abnormalities in the Fmr1 KO2 Mouse Model of Fragile X Syndrome")
tagger.predict(sentence)

In [131]:
type(sentence)

flair.data.Sentence

In [60]:
for keys in sentence.annotation_layers.keys():
    print(sentence.annotation_layers["key"])
    for e in entity.labels:
        print(e,entity.text)

Span[0:2]: "Behavioral abnormalities" → Disease (0.6736)


AttributeError: 'Label' object has no attribute 'labels'

In [100]:
for key in sentence.annotation_layers.keys():
    print(sentence.get_label())


Span[0:2]: "Behavioral abnormalities" → Disease (0.6736)
Span[0:2]: "Behavioral abnormalities" → Disease (0.6736)
Span[0:2]: "Behavioral abnormalities" → Disease (0.6736)


In [105]:
for la in sentence.get_labels():
    print(la.value,la.data_point.text,la.data_point.start_position,la.data_point.end_position)

Disease Behavioral abnormalities 0 24
Gene Fmr1 32 36
Species Mouse 41 46
Disease Fragile X Syndrome 56 74


('Disease', 'Behavioral abnormalities')

In [63]:
import flair

In [9]:
print(sentence.to_dict()["all labels"][-1]["value"])

Species


In [8]:
import sys

In [9]:
sys.path.append("/home/aberhe/Projects/SANTAL/ASLR/")

In [10]:
import src.artcileDetail_xml as ad

In [113]:
file_xml='/data/projects/santal/data/PMC_Articles/oa_comm/xml/all/PMC10000167.xml'#PMC10000014.xml'
with open(file_xml, 'r') as f:
    data = f.read()

In [114]:
articleDetails=ad.ArticleDetails(file_xml)
articleDetails.getDetails()

FILE READ!!


In [115]:
articleDetails.abstract

{'Simple Summary': 'Incorporating warm-season grasses into traditional cool-season grass equine rotational grazing systems can increase pasture availability during hot, dry months and bridge the “summer slump” forage gap. The objective of this study was to evaluate the impacts of this pasture management practice on the equine microbiome and to explore relationships between the fecal microbiota, forage nutrients, and metabolic responses of grazing horses. Results of this study indicate that distinct changes in microbial community structure and composition occur as horses adapt to different forages and that shifts in the microbial community were most influenced by forage non-structural carbohydrates and crude protein, rather than fiber. Interrelationships were found between these nutrients, glycemic responses, and Akkermansia and Clostridium butyricum. These bacteria were also found to be enriched in horses adapted to warm-season grasses. While the results of this study suggest that inte

In [128]:
len(sentence)

46

In [39]:
articleDetails.abstract

{'Simple Summary': 'The capital shortage for marine ranch construction will reduce their economic, ecological, and social benefits. By constructing the supply chain decision model to solve and analyze the results, we found that the change of the products’ environmental friendliness and the environmental enrichment degree of the marine ranching companies will affect the marine ranching construction level, and they are positively correlated. In addition, the enhancement of the guiding effect of government guidance fund can break through the dilemma of enterprise capital shortage and optimize the establishment and development level of marine ranching.\n',
 'Abstract': 'The construction of marine ranching is a concrete practice to fulfil the strategic objective of China’s maritime power. The shortage of funds has turned into an important issue to be resolved urgently in the modernization of marine ranching. This study constructs a supply chain system, involving a leading enterprise of mari

In [45]:
articleDetails.__dict__.keys()

dict_keys(['metadata', 'title', 'abstract', 'keywords', 'sections', 'references', 'soup'])

In [154]:
sentences = nltk.sent_tokenize(articleDetails.sections[key])
sentences=Sentence(sentences)

In [149]:
sentences=tagger.predict(sentences)

In [153]:
print(sentences)

None


In [116]:
# initialize the sentence splitter
#splitter = SciSpacySentenceSplitter()
for key in articleDetails.sections.keys():
    sentences = nltk.sent_tokenize(articleDetails.sections[key])
    for sentence in sentences:
        #print(sentence.replace("\n","").strip())
        sentence1=sentence.replace("\n","").strip()
        sentence = Sentence(sentence1)
        tagger.predict(sentence)
        if sentence.get_labels()!=[]:
            print(sentence1)
            for entity in sentence.get_labels():
                print(entity)
            print("*"*20)
    break

Warm- and cool-season grasses have different mechanisms for storage of soluble carbohydrates [4], with non-structural carbohydrate (NSC = sugars + starch + fructans) concentrations in cool-season grasses typically greater than that of warm-season grasses [1,5].
Span[12:13]: "carbohydrates" → Chemical (0.9907)
Span[19:20]: "carbohydrate" → Chemical (0.9919)
Span[23:24]: "sugars" → Chemical (0.8969)
********************
These differences in NSC content are of interest in equine management as current feeding recommendations for horses with existing metabolic dysfunction include limiting dietary NSC concentrations [6,7,8].
Span[9:10]: "equine" → Species (0.7423)
Span[16:17]: "horses" → Species (0.9682)
Span[19:21]: "metabolic dysfunction" → Disease (0.7248)
Span[24:25]: "NSC" → Chemical (0.661)
********************
Feeding supplemental concentrate higher in NSC has been shown to lower insulin sensitivity in horses [10,11,12], but over the relatively smaller range of NSC concentrations obse

In [3]:
def loadArticle(filePath):
    f=open(filePath)
    jsonData=json.load(f)
    
    return jsonData

In [4]:
def flairNER(sentences,tagger):
    sentence = Sentence(sentences)
    # run NER over sentence
    tagger.predict(sentence)
    
    return sentence

In [5]:
# load the NER tagger: just once
#tagger = SequenceTagger.load('hunflair-disease')
mTagger=MultiTagger.load('hunflair')

2023-01-16 13:54:22,069 loading file C:\Users\aberhe\.flair\models\hunflair-celline-v1.0.pt
2023-01-16 13:54:31,557 loading file C:\Users\aberhe\.flair\models\hunflair-chemical-full-v1.0.pt
2023-01-16 13:54:45,461 loading file C:\Users\aberhe\.flair\models\hunflair-disease-full-v1.0.pt
2023-01-16 13:55:05,041 loading file C:\Users\aberhe\.flair\models\hunflair-gene-full-v1.0.pt
2023-01-16 13:55:19,577 loading file C:\Users\aberhe\.flair\models\hunflair-species-full-v1.1.pt


In [2]:
MODEL_NAMES=['disease','cell-line','chemicals','gene','species']

In [ ]:
print('The aetiology of inflammatory bowel diseases (IBD) seems to be strongly connected to changes in the enteral microbiome. The dysbiosis pattern seen in Crohn’s disease (CD) differs among published studies depending on patients’ age, disease phenotype and microbiome research methods. The aims was to investigate microbiome in treatment-naive paediatric patients to get an insight into its structure at the early stage of the disease in comparison to healthy. Stool samples were obtained from controls and newly diagnosed patients prior to any intervention. Microbiota was analysed by 16SrRNAnext-generation-sequencing (NGS). Differences in the within-sample phylotype richness and evenness (alpha diversity) were detected between controls and patients. Statistically significant dissimilarities between samples were present for all used metrics. We also found a significant increase in the abundance of OTUs of the Enterococcus genus and reduction in, among others, Bifidobacterium (B. adolescentis), Roseburia (R.faecis), Faecalibacterium (F. prausnitzii), Gemmiger (G. formicilis), Ruminococcus (R. bromii) and Veillonellaceae (Dialister). Moreover, differences in alpha and beta diversities in respect to calprotectin and PCDAI were observed: patients with calprotectin <100\u2009µg/g and with PCDAI below 10 points vs those with calprotectin >100\u2009µg/g and mild (10–27.7 points), moderate (27.5–40 points) or severe (>40 points) CD disease activity had higher richness and diversity of gut microbiota. The results of our study highlight reduced diversity and dysbiosis at the earliest stage of the disease. Microbial imbalance and low abundance of butyrate-producing bacteria, including Bifidobacterium adolescentis, may suggest benefits of microbial modification therapy.')

In [6]:
sentence = Sentence('The aetiology of inflammatory bowel diseases (IBD) seems to be strongly connected to changes in the enteral microbiome. The dysbiosis pattern seen in Crohn’s disease (CD) differs among published studies depending on patients’ age, disease phenotype and microbiome research methods. The aims was to investigate microbiome in treatment-naive paediatric patients to get an insight into its structure at the early stage of the disease in comparison to healthy. Stool samples were obtained from controls and newly diagnosed patients prior to any intervention. Microbiota was analysed by 16SrRNAnext-generation-sequencing (NGS). Differences in the within-sample phylotype richness and evenness (alpha diversity) were detected between controls and patients. Statistically significant dissimilarities between samples were present for all used metrics. We also found a significant increase in the abundance of OTUs of the Enterococcus genus and reduction in, among others, Bifidobacterium (B. adolescentis), Roseburia (R.faecis), Faecalibacterium (F. prausnitzii), Gemmiger (G. formicilis), Ruminococcus (R. bromii) and Veillonellaceae (Dialister). Moreover, differences in alpha and beta diversities in respect to calprotectin and PCDAI were observed: patients with calprotectin <100\u2009µg/g and with PCDAI below 10 points vs those with calprotectin >100\u2009µg/g and mild (10–27.7 points), moderate (27.5–40 points) or severe (>40 points) CD disease activity had higher richness and diversity of gut microbiota. The results of our study highlight reduced diversity and dysbiosis at the earliest stage of the disease. Microbial imbalance and low abundance of butyrate-producing bacteria, including Bifidobacterium adolescentis, may suggest benefits of microbial modification therapy.')

# load the NER tagger
#tagger = SequenceTagger.load('hunflair-disease')
tagger=MultiTagger.load("hunflair")

# run NER over sentence
tagger.predict(sentence)

2023-01-16 13:57:22,684 loading file C:\Users\aberhe\.flair\models\hunflair-celline-v1.0.pt
2023-01-16 13:57:29,606 loading file C:\Users\aberhe\.flair\models\hunflair-chemical-full-v1.0.pt
2023-01-16 13:57:38,916 loading file C:\Users\aberhe\.flair\models\hunflair-disease-full-v1.0.pt
2023-01-16 13:57:52,219 loading file C:\Users\aberhe\.flair\models\hunflair-gene-full-v1.0.pt
2023-01-16 13:58:01,269 loading file C:\Users\aberhe\.flair\models\hunflair-species-full-v1.1.pt


c:\ProgramData\Anaconda3\envs\qenv\lib\site-packages\flair\models\sequence_tagger_model.py:1046: UserWarning: We recommend to use SciSpaCy for tokenization and sentence splitting if HunFlair is applied to biomedical text, e.g.

from flair.tokenization import SciSpacySentenceSplitter
sentence = Sentence('Your biomed text', use_tokenizer=SciSpacySentenceSplitter())

  warn(


In [16]:
print(sentence.to_tagged_string())

The aetiology of inflammatory <B-Disease> bowel <I-Disease> diseases <E-Disease> ( IBD <S-Disease> ) seems to be strongly connected to changes in the enteral microbiome . The dysbiosis pattern seen in Crohn <B-Disease> ’s <I-Disease> disease <E-Disease> ( CD <S-Disease> ) differs among published studies depending on patients’ age , disease phenotype and microbiome research methods . The aims was to investigate microbiome in treatment-naive paediatric patients to get an insight into its structure at the early stage of the disease in comparison to healthy . Stool samples were obtained from controls and newly diagnosed patients prior to any intervention . Microbiota was analysed by 16SrRNAnext-generation-sequencing ( NGS ) . Differences in the within-sample phylotype richness and evenness ( alpha diversity ) were detected between controls and patients . Statistically significant dissimilarities between samples were present for all used metrics . We also found a significant increase in the

In [9]:
#Load Articles from the following Path
pathArticles=r"C:\\Users\\aberhe\\Desktop\\PostDoc\\Projects\\SR\\SR project\\downloadedArticles\\Included Articles\\Cirrhosis\\Veillonella"

folder=r"Alistipes"
file=pathArticles+'\\'+os.listdir(pathArticles)[2]

#A file to the predomics papers
file=r'C:\\Users\\aberhe\\Desktop\\PostDoc\\Projects\\SANTAL\\codes\\Article_Extraction\\veillonella\\PMC7062144.json'

In [10]:
file_Veillonella=os.listdir(pathArticles)
len(file_Veillonella)

9

In [ ]:
fileList=[os.path.join(pathArticles,file) for file in file_Veillonella]

In [5]:
def getEntitiesSpaciesDisease(filesList=None,part_of_article='full',output_file="All_Vellonella_Articles"):
    df=pd.DataFrame(columns=["text","entity1","entity2","joined_text"])
    if filesList==None:
        return None
    else:
        for file in filesList:
            jsonData=loadArticle(file)
            artDet=ArticleDetails(jsonData)
            artDet.getDetails() 
            
            if part_of_article=='full':
                absSent=artDet.fullArticle.strip()
            
            elif part_of_article=='abstract':
                absSent=artDet.abstract.strip()
            
            absSent=nltk.sent_tokenize(absSent)
            for sent in tqdm(absSent): 
                nerAbs=flairNER(sent,tagger=mTagger)
                chemicals=nerAbs.to_dict('hunflair-chemical')
                disease=nerAbs.to_dict('hunflair-disease')
                spacies=nerAbs.to_dict('hunflair-species')
                genes=nerAbs.to_dict('hunflair-gene')
                cell_line=nerAbs.to_dict('hunflair-celline')
                
                s=[e["text"] if spacies["entities"]!=[] else [] for e in spacies["entities"]]
                d=[e["text"] if disease["entities"]!=[] else [] for e in disease["entities"] ]
                c=[e["text"] if chemicals["entities"]!=[] else [] for e in chemicals["entities"]]
                g=[e["text"] if genes["entities"]!=[] else [] for e in genes["entities"]]
                l=[e["text"] if cell_line["entities"]!=[] else [] for e in cell_line["entities"]]
                
                pairs_d_s=[[a, b] for a in d for b in s]
                
                pairs_c_s=[[a, b] for a in c for b in s]
                
                pairs_g_s=[[a, b] for a in g for b in s]
                
                pairs_l_s=[[a, b] for a in l for b in s]
                
                all_pairs=pairs_d_s+pairs_c_s+pairs_g_s+pairs_l_s
                
                for pairs in all_pairs:
                    joined_text=pairs[1]+" , "+ sent+" , "+pairs[0]
                    df.loc[len(df)]=[sent,pairs[1],pairs[0],joined_text]
        df.to_csv(output_file+".csv")
        return df

In [95]:
df_Veillonella=getEntitiesSpaciesDisease(filesList=fileList)

100%|████████████████████████████████████████████████████████████████████████████████| 163/163 [03:52<00:00,  1.42s/it]


In [8]:
jsonData=loadArticle(file)

artDet=ArticleDetails(jsonData)

artDet.getDetails()

NameError: name 'file' is not defined

In [59]:
list(set([]))

[]

In [27]:
print(f'The title of the article is: \n{artDet.title.strip()}')

The title of the article is: 
Interpretable and accurate prediction models for metagenomics data


In [72]:
nerAbs=flairNER("Alterations of the human gut microbiome in liver cirrhosis and cancer",tagger=mTagger)
chemicals=nerAbs.to_dict('hunflair-chemical')
disease=nerAbs.to_dict('hunflair-disease')
spacies=nerAbs.to_dict('hunflair-species')
genes=nerAbs.to_dict('hunflair-gene')
cell_line=nerAbs.to_dict('hunflair-celline')

In [74]:
d=[e["text"] for e in disease["entities"] if disease["entities"]!=[]]
s=[e["text"] for e in spacies["entities"] if spacies["entities"]!=[]]

In [ ]:
s=[e["text"] if spacies["entities"]!=[] else [] for e in enumerate(spacies["entities"]) ]
d=[e["text"] if disease["entities"]!=[] else [] for e in enumerate(disease["entities"]) ]
c=[e["text"] if chemicals["entities"]!=[] else [] for e in enumerate(chemicals["entities"])]
g=[e["text"] if genes["entities"]!=[] else [] for e in enumerate(genes["entities"]) ]
l=[e["text"] if cell_line["entities"]!=[] else [] for e in enumerate(cell_line["entities"])]

In [64]:
output = [[a, b] for a in [] for b in s]

In [65]:
output

[]

In [13]:
print(f'The abstract of the article is: \n{artDet.abstract.strip()}')

The title of the article is: 
Microbiome biomarker discovery for patient diagnosis, prognosis, and risk evaluation is attracting broad interest. Selected groups of microbial features provide signatures that characterize host disease states such as cancer or cardio-metabolic diseases. Yet, the current predictive models stemming from machine learning still behave as black boxes and seldom generalize well. Their interpretation is challenging for physicians and biologists, which makes them difficult to trust and use routinely in the physician–patient decision-making process. Novel methods that provide interpretability and biological insight are needed. Here, we introduce “predomics”, an original machine learning approach inspired by microbial ecosystem interactions that is tailored for metagenomics data. It discovers accurate predictive signatures and provides unprecedented interpretability. The decision provided by the predictive model is based on a simple, yet powerful score computed by 

In [36]:
#absSent=artDet.fullArticle.strip()
absSent="opportunistic pathogens like Megasphaera micronuciformis that proliferate in patients with liver cirrhosis."

In [37]:
absSent=nltk.sent_tokenize(absSent)

In [30]:
absSent

['opportunistic pathogens like Megasphaera micronuciformis that proliferate in patients with liver cirrhosis.']

In [39]:
nerAbs=flairNER(absSent[0],tagger=mTagger)

C:\ProgramData\Anaconda3\envs\qenv\lib\site-packages\flair\models\sequence_tagger_model.py:1046: UserWarning: We recommend to use SciSpaCy for tokenization and sentence splitting if HunFlair is applied to biomedical text, e.g.

from flair.tokenization import SciSpacySentenceSplitter
sentence = Sentence('Your biomed text', use_tokenizer=SciSpacySentenceSplitter())

  warn(


In [60]:
for token in nerAbs.tokens:
    print(token.text,end="  ")

opportunistic  pathogens  like  Megasphaera  micronuciformis  that  proliferate  in  patients  with  liver  cirrhosis  .  

In [54]:
nerAbs.to_tagged_string()

'opportunistic pathogens like Megasphaera <B-Species> micronuciformis <E-Species> that proliferate in patients with liver <B-Disease> cirrhosis <E-Disease> .'

In [47]:
type(nerAbs.tokens[0].text)

str

In [35]:
nerAbs.tokens
print(nerAbs.tokens)

[Token: 1 The, Token: 2 best, Token: 3 Ratio, Token: 4 and, Token: 5 Ter, Token: 6 models, Token: 7 (, Token: 8 1, Token: 9 –, Token: 10 3, Token: 11 ), Token: 12 include, Token: 13 oral, Token: 14 bacterial, Token: 15 species, Token: 16 of, Token: 17 the, Token: 18 genus, Token: 19 Veillonella, Token: 20 (, Token: 21 Veillonellaunclassified, Token: 22 ), Token: 23 ,, Token: 24 Streptococcus, Token: 25 (, Token: 26 S., Token: 27 parasanguinis, Token: 28 and, Token: 29 S., Token: 30 anginosus, Token: 31 ), Token: 32 ,, Token: 33 and, Token: 34 opportunistic, Token: 35 pathogens, Token: 36 like, Token: 37 Megasphaera, Token: 38 micronuciformis, Token: 39 that, Token: 40 proliferate, Token: 41 in, Token: 42 patients, Token: 43 with, Token: 44 liver, Token: 45 cirrhosis, Token: 46 ,, Token: 47 whereas, Token: 48 butyrate, Token: 49 producers, Token: 50 of, Token: 51 the, Token: 52 genus, Token: 53 Subdoligranilum, Token: 54 (, Token: 55 Subdoligranilumunclassified, Token: 56 ), Token: 57 c

In [2]:
#print(absSent)

In [13]:
chemicals=[]
disease=[]
genes=[]
spacies=[]
cell_line=[]
for sent in absSent: 
    nerAbs=flairNER(sent,tagger=mTagger)
    print(nerAbs.to_tagged_string())
    print(nerAbs.annotation_layers.keys())
    print(nerAbs.to_dict('hunflair-species'))
    chemicals.append(nerAbs.to_dict('hunflair-chemical'))
    disease.append(nerAbs.to_dict('hunflair-disease'))
    spacies.append(nerAbs.to_dict('hunflair-species'))
    genes.append(nerAbs.to_dict('hunflair-gene'))
    cell_line.append(nerAbs.to_dict('hunflair-celline'))
    #print(nerAbs.to_dict('hunflair-disease'))
    print("-------->NER<------")
    for annLayer in nerAbs.annotation_layers.keys():
        print("Here: ",annLayer)
        for entity in nerAbs.get_spans(annLayer):
            print(entity)

C:\ProgramData\Anaconda3\envs\qenv\lib\site-packages\flair\models\sequence_tagger_model.py:1046: UserWarning: We recommend to use SciSpaCy for tokenization and sentence splitting if HunFlair is applied to biomedical text, e.g.

from flair.tokenization import SciSpacySentenceSplitter
sentence = Sentence('Your biomed text', use_tokenizer=SciSpacySentenceSplitter())

  warn(


The best Ratio and Ter models ( 1 – 3 ) include oral bacterial species of the genus Veillonella ( Veillonellaunclassified ) , Streptococcus <S-Species> ( S. <B-Species> parasanguinis <E-Species> and S. <B-Species> anginosus <E-Species> ) , and opportunistic pathogens like Megasphaera <B-Species> micronuciformis <E-Species> that proliferate in patients with liver <B-Disease> cirrhosis <E-Disease> , whereas butyrate <S-Chemical> producers of the genus Subdoligranilum ( Subdoligranilumunclassified ) closely related to Faecalibacterium <B-Species> prausnitzii <E-Species> and complex polysaccharide-degrading species like Bacteroides <B-Species> cellulosilyticus <E-Species> characterize control subjects .
dict_keys([])
{'text': 'The best Ratio and Ter models (1–3) include oral bacterial species of the genus Veillonella (Veillonellaunclassified), Streptococcus (S. parasanguinis and S. anginosus), and opportunistic pathogens like Megasphaera micronuciformis that proliferate in patients with li

In [14]:
s=[se for i, se in enumerate(spacies) if se["entities"]!=[]]
d=[de for i, de in enumerate(disease) if de["entities"]!=[]]
c=[de for i, de in enumerate(chemicals) if de["entities"]!=[]]
g=[de for i, de in enumerate(genes) if de["entities"]!=[]]

In [15]:
s[0]

{'text': 'The best Ratio and Ter models (1–3) include oral bacterial species of the genus Veillonella (Veillonellaunclassified), Streptococcus (S. parasanguinis and S. anginosus), and opportunistic pathogens like Megasphaera micronuciformis that proliferate in patients with liver cirrhosis, whereas butyrate producers of the genus Subdoligranilum (Subdoligranilumunclassified) closely related to Faecalibacterium prausnitzii and complex polysaccharide-degrading species like Bacteroides cellulosilyticus characterize control subjects.',
 'labels': [],
 'entities': [{'text': 'Streptococcus',
   'start_pos': 119,
   'end_pos': 132,
   'labels': [Species (0.7683)]},
  {'text': 'S. parasanguinis',
   'start_pos': 134,
   'end_pos': 150,
   'labels': [Species (0.7893)]},
  {'text': 'S. anginosus',
   'start_pos': 155,
   'end_pos': 167,
   'labels': [Species (0.7591)]},
  {'text': 'Megasphaera micronuciformis',
   'start_pos': 203,
   'end_pos': 230,
   'labels': [Species (0.8325)]},
  {'text': 

In [16]:
ch_ext=[abc['text'] for ab in c for abc in ab['entities']]
gn_ext=[abc['text'] for ab in g for abc in ab['entities']]
sp_ext=[abc['text'] for ab in s for abc in ab['entities']]
di_ext=[abc['text'] for ab in d for abc in ab['entities']]

In [17]:
sp_ext

['Streptococcus',
 'S. parasanguinis',
 'S. anginosus',
 'Megasphaera micronuciformis',
 'Faecalibacterium prausnitzii',
 'Bacteroides cellulosilyticus']

In [18]:
di_ext

['liver cirrhosis']

In [19]:
gn_ext

[]

In [20]:
ch_ext

['butyrate']

In [47]:
def get_EntityPairs(entity1,entity2):
    e1=[se for i, se in enumerate(entity1) if se["entities"]!=[]]
    e2=[de for i, de in enumerate(entity2) if de["entities"]!=[]]
    
    e1_ext=[[ent1_only['text'],ent1['text']] for ent1 in e1 for ent1_only in ent1['entities']]
    e2_ext=[[ent2_only['text'],ent2['text']] for ent2 in e2 for ent2_only in ent2['entities']]
    
    return e1_ext,e2_ext

In [100]:
e1_ext,e2_ext=get_EntityPairs(spacies,disease)

In [95]:
len(e1_ext),len(e2_ext)

(8, 2)

In [101]:
e2_ext

[['disease',
  'In a linear model, a patient is predicted in a disease group with a probability of p > 0.5 if , where  are real coefficients.'],
 ['Cirrhosis',
  'When learning from the different types of variables based on taxonomic levels (Cirrhosis Stage 1), the performance of the models varies accordingly.'],
 ['disease',
  'Indeed, in this space, models can be powerful because they can summarize more complex rules such as: “if (abundance of all Firmicutes − abundance of all Clostridiales order) > threshold then disease.” We tested the generalization of Bin, Ter, Ratio and also TerLog models trained in Cirrhosis Stage 1, in a second, independent dataset (i.e., Cirrhosis Stage 2).'],
 ['Cirrhosis',
  'Indeed, in this space, models can be powerful because they can summarize more complex rules such as: “if (abundance of all Firmicutes − abundance of all Clostridiales order) > threshold then disease.” We tested the generalization of Bin, Ter, Ratio and also TerLog models trained in Cir

In [108]:
df=pd.DataFrame(columns=["text","entity1","entity2","joined_text"])

In [109]:
for e1 in e1_ext:
    for e2 in e2_ext:
        if e1[1]==e2[1]:
            if "Veillonella" in e1[1]:
                joined_text="Veillonella"+" , "+e1[1]+ " , "+e2[0]
                df.loc[len(df)]=[e1[1],"Veillonella",e2[0],joined_text]
            joined_text=e1[0]+" , "+e1[1]+ " , "+e2[0]
            df.loc[len(df)]=[e1[1],e1[0],e2[0],joined_text]

In [111]:
df

,text,entity1,entity2,joined_text
0,The potential competition between oral and gut...,Veillonella,liver cirrhosis,"Veillonella , The potential competition betwee..."
1,The potential competition between oral and gut...,S9,liver cirrhosis,"S9 , The potential competition between oral an..."
2,The best Ratio and Ter models (1–3) include or...,Veillonella,liver cirrhosis,"Veillonella , The best Ratio and Ter models (1..."
3,The best Ratio and Ter models (1–3) include or...,Streptococcus,liver cirrhosis,"Streptococcus , The best Ratio and Ter models ..."
4,The best Ratio and Ter models (1–3) include or...,Veillonella,liver cirrhosis,"Veillonella , The best Ratio and Ter models (1..."
5,The best Ratio and Ter models (1–3) include or...,S. parasanguinis,liver cirrhosis,"S. parasanguinis , The best Ratio and Ter mode..."
6,The best Ratio and Ter models (1–3) include or...,Veillonella,liver cirrhosis,"Veillonella , The best Ratio and Ter models (1..."
7,The best Ratio and Ter models (1–3) include or...,S. anginosus,liver cirrhosis,"S. anginosus , The best Ratio and Ter models (..."
8,The best Ratio and Ter models (1–3) include or...,Veillonella,liver cirrhosis,"Veillonella , The best Ratio and Ter models (1..."
9,The best Ratio and Ter models (1–3) include or...,Megasphaera micronuciformis,liver cirrhosis,"Megasphaera micronuciformis , The best Ratio a..."


In [114]:
df.text[2]

'The best Ratio and Ter models (1–3) include oral bacterial species of the genus Veillonella(Veillonellaunclassified), Streptococcus(S. parasanguinis and S. anginosus), and opportunistic pathogens like Megasphaera micronuciformis that proliferate in patients with liver cirrhosis, whereas butyrate producers of the genus Subdoligranilum (Subdoligranilumunclassified) closely related to Faecalibacterium prausnitzii and complex polysaccharide-degrading species like Bacteroides cellulosilyticus characterize control subjects.'

In [113]:
df.to_csv("Predomics_Entities.csv")

In [15]:
len(absSent),len(chemicals)

(11, 11)

In [25]:
genes

[{'text': 'Microbiome biomarker discovery for patient diagnosis, prognosis, and risk evaluation is attracting broad interest.',
  'labels': [],
  'entities': []},
 {'text': 'Selected groups of microbial features provide signatures that characterize host disease states such as cancer or cardio-metabolic diseases.',
  'labels': [],
  'entities': []},
 {'text': 'Yet, the current predictive models stemming from machine learning still behave as black boxes and seldom generalize well.',
  'labels': [],
  'entities': []},
 {'text': 'Their interpretation is challenging for physicians and biologists, which makes them difficult to trust and use routinely in the physician–patient decision-making process.',
  'labels': [],
  'entities': []},
 {'text': 'Novel methods that provide interpretability and biological insight are needed.',
  'labels': [],
  'entities': []},
 {'text': 'Here, we introduce “predomics”, an original machine learning approach inspired by microbial ecosystem interactions that is

In [25]:
disease[0]['entities'][0]['text']

'inflammatory bowel diseases'

In [50]:
for dea in disease:
    if len(dea['entities'])>0:
        for j in range(len(dea['entities'])):
            print(dea['entities'][j]['text'])

primary sclerosing cholangitis
psc
cholestatic liver disease
biliary inflammation
psc
inflammatory bowel disease
psc
psc
psc


In [27]:
for i in range(11):
    print('chemical: {} disease: {} gene: {} spacies: {}'.format(chemicals[i]['entities'][0]['text'],disease[i]['entities'][0]['text'],genes[i]['entities'][0]['text'],spacies[i]['entities'][0]['text']))
    print()

IndexError: list index out of range

In [16]:
fullArt=artDet.fullArticle
fullArtSent=nltk.sent_tokenize(fullArt)

In [17]:
chemicals=[]
disease=[]
genes=[]
spacies=[]
for sent in fullArtSent: 
    nerAbs=flairNER(sent)
    print(nerAbs.to_tagged_string())
    chemicals.append(nerAbs.to_dict('hunflair-chemical'))
    disease.append(nerAbs.to_dict('hunflair-disease'))
    spacies.append(nerAbs.to_dict('hunflair-spacies'))
    genes.append(nerAbs.to_dict('hunflair-gene'))
    #print(nerAbs.to_dict('hunflair-disease'))
    print("-------->NER<------")
    for annLayer in nerAbs.annotation_layers.keys():
        print("Here: ",annLayer)
        for entity in nerAbs.get_spans(annLayer):
            print(entity)

C:\ProgramData\Anaconda3\envs\qenv\lib\site-packages\flair\models\sequence_tagger_model.py:1046: UserWarning: We recommend to use SciSpaCy for tokenization and sentence splitting if HunFlair is applied to biomedical text, e.g.

from flair.tokenization import SciSpacySentenceSplitter
sentence = Sentence('Your biomed text', use_tokenizer=SciSpacySentenceSplitter())

  warn(


Inflammatory <B-Disease> bowel <I-Disease> disease <E-Disease> ( IBD <S-Disease> ) is a chronic progressive and idiopathic <B-Disease> inflammatory <I-Disease> disorder <E-Disease> of the gastrointestinal tract ( GI ) tract , which includes ulcerative <B-Disease> colitis <E-Disease> ( UC <S-Disease> ) and Crohn <B-Disease> ’s <I-Disease> disease <E-Disease> ( CD <S-Disease> ) .
-------->NER<------
Although the etiology of IBD <S-Disease> remains largely unknown , it involves a complex interaction between the genetic , environmental , and aberrant immune responses .
-------->NER<------
In recent times , the involvement of gut microbiota is considered a new probable factor strongly connected with IBD <S-Disease> pathogenesis .
-------->NER<------
However , it is still unclear whether this dysbiosis is a primary or secondary event in the relationship with IBD <S-Disease> .
-------->NER<------
In this regard , Tomasello et al .
-------->NER<------
reported several mechanisms that are respo

In [20]:
len(genes)

203

In [33]:
sentence=[]
entities=[]
for i in range(len(chemicals)):
    if genes[i]['entities']!=[] and disease[i]['entities']!=[]:
        entyTemp=[]
        for j in range(len(disease[i]['entities'])):
            entyTemp.append((disease[i]['entities'][j]['text'],genes[i]['entities'][0]['text']))
        sentence.append(genes[i]['text'])
        entities.append(entyTemp)

In [34]:
for sent, ent in zip(sentence,entities):
    print(sent)
    print(ent)

In order to monitor the patient’s clinical features, routine diagnostic blood tests were performed including a hemoglobin count (Hb (g/L)), white cell count (WBC (109/L)), red blood count (RBC (1012/L)), platelet count (PLT (109/L)), markers of inflammation such as C reactive protein (CRP (mg/dL)), iron (Fe (ug/dL)), total iron-binding capacity (TIBC (ug/dL)), serum ferritin (ug/L) as an indicator of iron storage, total protein level (TP g/dL), and albumin (g/dL).
[('inflammation', 'hemoglobin')]
Additionally, fecal calprotectin as a non-invasive test for the direct evaluation of intestinal inflammation was measured using a commercially available quantitative enzyme-linked immunoassay (PhiCal Calprotectin Elisa Kit, Immunodiagnostic, Bensheim, Germany).
[('inflammation', 'calprotectin')]
In the intestine, F. prausnitzii produces butyrate, which can reduce intestinal mucosa inflammation by inhibiting the nuclear factor kappa-light-chain-enhancer of activated B cells (NF-κB) transcriptio

In [15]:
from flair.tokenization import SciSpacySentenceSplitter, SciSpacyTokenizer

In [16]:
sentSpliter=SciSpacySentenceSplitter()

In [17]:
sentencesSplited=sentSpliter.split(artDet.abstract)

In [25]:
print(sentencesSplited)

[Sentence: "  Consumption of alcohol ( ethanol ) in various forms has been an integral part of human civilization ."   [− Tokens: 19  − Token-Labels: "  Consumption of alcohol <S-Chemical> ( ethanol <S-Chemical> ) in various forms has been an integral part of human <S-Species> civilization ."], Sentence: "Since ages , it also has been an important cause of death and health impairment across the globe ."   [− Tokens: 19], Sentence: "Ethanol - mediated liver injury , known as alcoholic liver disease ( ALD ) , is caused by surplus intake of alcohol ."   [− Tokens: 23  − Token-Labels: "Ethanol <S-Chemical> - mediated liver <B-Disease> injury <E-Disease> , known as alcoholic <B-Disease> liver <I-Disease> disease <E-Disease> ( ALD <S-Disease> ) , is caused by surplus intake of alcohol <S-Chemical> ."], Sentence: "Several studies have proposed the different pathways that may be lead to ALD ."   [− Tokens: 14  − Token-Labels: "Several studies have proposed the different pathways that may be le

In [19]:
print(absSent)

['Consumption of alcohol (ethanol) in various forms has been an integral part of human civilization.', 'Since ages, it also has been an important cause of death and health impairment across the globe.', 'Ethanol-mediated liver injury, known as alcoholic liver disease (ALD), is caused by surplus intake of alcohol.', 'Several studies have proposed the different pathways that may be lead to ALD.', 'One of the factors that may affect the cytochrome P450 (CYP2E1) metabolic pathway is gut dysbiosis.', 'The gut microbiota produces various compounds that play an important role in regulating healthy functions of distal organs such as the adipose tissue and liver.', 'Dysbiosis causes bacteremia, hepatic encephalopathy, and increased intestinal permeability.', 'Recent clinical studies have found better understanding of the gut and liver axis.', 'Another factor that may affect the ALD pathway is dysfunction of adipose tissue metabolism.', 'Moreover, dysfunction of adipose tissue leads to ectopic f

In [21]:
tagger=MultiTagger.load('hunflair')

2022-08-02 14:19:24,016 loading file C:\Users\aberhe\.flair\models\hunflair-celline-v1.0.pt
2022-08-02 14:19:33,492 loading file C:\Users\aberhe\.flair\models\hunflair-chemical-full-v1.0.pt
2022-08-02 14:19:41,411 loading file C:\Users\aberhe\.flair\models\hunflair-disease-full-v1.0.pt
2022-08-02 14:19:50,437 loading file C:\Users\aberhe\.flair\models\hunflair-gene-full-v1.0.pt
2022-08-02 14:19:59,090 loading file C:\Users\aberhe\.flair\models\hunflair-species-full-v1.1.pt


In [22]:
tagger.predict(sentencesSplited)

In [126]:
sents.to_original_text()

'This review is aimed at the effect of ethanol on gut and adipose tissue in both rodent and human alcoholic models.'

In [119]:
print(i.text,i.tag,round(i.score,3))

human Species 0.965


In [150]:
jsonDataL=[]
for sentTag in sentencesSplited:
    s=sentTag.to_original_text()
    jsonData={"setence":s,"entities":[]}
    entityDicL=[]
    for spanTag in sentTag.get_spans():
        #taggedEntities=[s,spanTag.text,spanTag.start_pos,spanTag.end_pos,spanTag.tag,round(spanTag.score,3)]
        entityDic={"text":spanTag.text,"start_chunk":spanTag.start_pos, "end_chunk":spanTag.end_pos,"label":spanTag.tag,"score":spanTag.score}
        entityDicL.append(entityDic)
        jsonData['entities'].append(entityDic)
    jsonDataL.append(jsonData)
with open('outputfile.json', 'w') as fout:
    json.dump(jsonDataL, fout)

In [151]:
with open('outputfile.json', 'r') as fout:
    jsonD=json.load(fout)

In [152]:
jsonD

[{'setence': ' Consumption of alcohol (ethanol) in various forms has been an integral part of human civilization.',
  'entities': [{'text': 'alcohol',
    'start_chunk': 16,
    'end_chunk': 23,
    'label': 'Chemical',
    'score': 0.974072277545929},
   {'text': 'ethanol',
    'start_chunk': 25,
    'end_chunk': 32,
    'label': 'Chemical',
    'score': 0.9896111488342285},
   {'text': 'human',
    'start_chunk': 80,
    'end_chunk': 85,
    'label': 'Species',
    'score': 0.9929239749908447}]},
 {'setence': 'Since ages, it also has been an important cause of death and health impairment across the globe.',
  'entities': []},
 {'setence': 'Ethanol-mediated liver injury, known as alcoholic liver disease (ALD), is caused by surplus intake of alcohol.',
  'entities': [{'text': 'Ethanol',
    'start_chunk': 0,
    'end_chunk': 7,
    'label': 'Chemical',
    'score': 0.9916825294494629},
   {'text': 'alcohol',
    'start_chunk': 102,
    'end_chunk': 109,
    'label': 'Chemical',
    'sc

In [143]:
jsonDataL

[{'text': ' Consumption of alcohol (ethanol) in various forms has been an integral part of human civilization.',
  'entities': [[{'text': 'alcohol',
     'start_chunk': 16,
     'end_chunk': 23,
     'label': 'Chemical',
     'score': 0.974072277545929},
    {'text': 'ethanol',
     'start_chunk': 25,
     'end_chunk': 32,
     'label': 'Chemical',
     'score': 0.9896111488342285},
    {'text': 'human',
     'start_chunk': 80,
     'end_chunk': 85,
     'label': 'Species',
     'score': 0.9929239749908447}]]},
 {'text': 'Since ages, it also has been an important cause of death and health impairment across the globe.',
  'entities': [[]]},
 {'text': 'Ethanol-mediated liver injury, known as alcoholic liver disease (ALD), is caused by surplus intake of alcohol.',
  'entities': [[{'text': 'Ethanol',
     'start_chunk': 0,
     'end_chunk': 7,
     'label': 'Chemical',
     'score': 0.9916825294494629},
    {'text': 'alcohol',
     'start_chunk': 102,
     'end_chunk': 109,
     'label': '

In [130]:
df=pd.DataFrame(columns=("sentence","entity","start_chunk","end_chunk","lable","score"))
for sents in sentencesSplited:
    s=sents.to_original_text()
    for i in sents.get_spans():
        print(s,i.text,i.start_pos,i.end_pos,i.tag,round(i.score,3))
        df.loc[len(df)]=[s,i.text,i.start_pos,i.end_pos,i.tag,round(i.score,3)]

 Consumption of alcohol (ethanol) in various forms has been an integral part of human civilization. alcohol 16 23 Chemical 0.974
 Consumption of alcohol (ethanol) in various forms has been an integral part of human civilization. ethanol 25 32 Chemical 0.99
 Consumption of alcohol (ethanol) in various forms has been an integral part of human civilization. human 80 85 Species 0.993
Ethanol-mediated liver injury, known as alcoholic liver disease (ALD), is caused by surplus intake of alcohol. Ethanol 0 7 Chemical 0.992
Ethanol-mediated liver injury, known as alcoholic liver disease (ALD), is caused by surplus intake of alcohol. alcohol 102 109 Chemical 0.951
Ethanol-mediated liver injury, known as alcoholic liver disease (ALD), is caused by surplus intake of alcohol. liver injury 17 29 Disease 0.732
Ethanol-mediated liver injury, known as alcoholic liver disease (ALD), is caused by surplus intake of alcohol. alcoholic liver disease 40 63 Disease 0.932
Ethanol-mediated liver injury, known a

In [131]:
df

,sentence,entity,start_chunk,end_chunk,lable,score
0,Consumption of alcohol (ethanol) in various f...,alcohol,16,23,Chemical,0.974
1,Consumption of alcohol (ethanol) in various f...,ethanol,25,32,Chemical,0.990
2,Consumption of alcohol (ethanol) in various f...,human,80,85,Species,0.993
3,"Ethanol-mediated liver injury, known as alcoho...",Ethanol,0,7,Chemical,0.992
4,"Ethanol-mediated liver injury, known as alcoho...",alcohol,102,109,Chemical,0.951
5,"Ethanol-mediated liver injury, known as alcoho...",liver injury,17,29,Disease,0.732
6,"Ethanol-mediated liver injury, known as alcoho...",alcoholic liver disease,40,63,Disease,0.932
7,"Ethanol-mediated liver injury, known as alcoho...",ALD,65,68,Disease,1.000
8,Several studies have proposed the different pa...,ALD,73,76,Disease,0.999
9,One of the factors that may affect the cytochr...,cytochrome P450,39,54,Gene,0.761


In [75]:
for sents in sentencesSplited:
    print(sents.to_tagged_string())
    #print("#"*30)
    set

  Consumption of alcohol <S-Chemical> ( ethanol <S-Chemical> ) in various forms has been an integral part of human <S-Species> civilization .
Since ages , it also has been an important cause of death and health impairment across the globe .
Ethanol <S-Chemical> - mediated liver <B-Disease> injury <E-Disease> , known as alcoholic <B-Disease> liver <I-Disease> disease <E-Disease> ( ALD <S-Disease> ) , is caused by surplus intake of alcohol <S-Chemical> .
Several studies have proposed the different pathways that may be lead to ALD <S-Disease> .
One of the factors that may affect the cytochrome <B-Gene> P450 <E-Gene> ( CYP2E1 <S-Gene> ) metabolic pathway is gut dysbiosis .
The gut microbiota produces various compounds that play an important role in regulating healthy functions of distal organs such as the adipose tissue and liver .
Dysbiosis <S-Disease> causes bacteremia <S-Disease> , hepatic <B-Disease> encephalopathy <E-Disease> , and increased intestinal permeability .
Recent clinical s

In [87]:
chemicals=[]
disease=[]
genes=[]
spacies=[]
hun=[]
for sent in sentencesSplited: 
    #nerAbs=flairNER(sent)
    #print(nerAbs.to_tagged_string())
    c=sent.to_dict('hunflair-chemical')['entities']
    d=sent.to_dict('hunflair-disease')['entities']
    g=sent.to_dict('hunflair-gene')['entities']
    s=sent.to_dict('hunflair-spacies')['entities']
    print(sent)
    if  c:
        print(c)
        print("we have chemical")
    elif  d:
        print(d)
        print("we have disease")
    elif  g:
        print(g)
        print("we have gene")
    elif  s:
        print(s)
        print("we have spacies")
    chemicals.append(sent.to_dict('hunflair-chemical'))
    disease.append(sent.to_dict('hunflair-disease'))
    spacies.append(sent.to_dict('hunflair-spacies'))
    genes.append(sent.to_dict('hunflair-gene'))

Sentence: "  Consumption of alcohol ( ethanol ) in various forms has been an integral part of human civilization ."   [− Tokens: 19  − Token-Labels: "  Consumption of alcohol <S-Chemical> ( ethanol <S-Chemical> ) in various forms has been an integral part of human <S-Species> civilization ."]
[{'text': 'alcohol', 'start_pos': 16, 'end_pos': 23, 'labels': [Chemical (0.9741)]}, {'text': 'ethanol', 'start_pos': 25, 'end_pos': 32, 'labels': [Chemical (0.9896)]}]
we have chemical
Sentence: "Since ages , it also has been an important cause of death and health impairment across the globe ."   [− Tokens: 19]
Sentence: "Ethanol - mediated liver injury , known as alcoholic liver disease ( ALD ) , is caused by surplus intake of alcohol ."   [− Tokens: 23  − Token-Labels: "Ethanol <S-Chemical> - mediated liver <B-Disease> injury <E-Disease> , known as alcoholic <B-Disease> liver <I-Disease> disease <E-Disease> ( ALD <S-Disease> ) , is caused by surplus intake of alcohol <S-Chemical> ."]
[{'text': 

In [100]:
chemicals[0]['entities'][0]['labels'][0].value

'Chemical'

In [54]:
def getMaxEntitiesLength(c,d,g,s):
    nerList=['chemicals','disease','genes','spacies']
    lenList=[len(c['entities']),len(d['entities']),len(g['entities']),len(s['entities'])]
    
    return nerList[lenList.index(max(lenList))]

In [49]:
len(chemicals[0]['entities']),len(sentencesSplited),len(disease[0]['entities']),len(spacies[0]['entities']),len(genes[0]['entities'])

(2, 14, 0, 0, 0)

In [ ]:
nerList=[chemicals[0],disease[0],genes[0],spacies[0]]

In [55]:
getMaxEntitiesLength(chemicals[0],disease[0],genes[0],spacies[0])

'chemicals'

In [ ]:
print("Sentnce \t Chemical \t Disease \t gene \t Spacies")
for i in range(len(chemicals)):
    for j in range(len(chemicals[i]))
    print(chemicals[i]+ stat)

In [158]:
disease

[{'text': 'An increasing wealth of data from high-throughput molecular and imaging technologies is connecting biomedical sciences and machine learning (ML).',
  'labels': [],
  'entities': []},
 {'text': 'The latter is affecting numerous areas of medicine, including disease diagnosis and prognosis.',
  'labels': [],
  'entities': []},
 {'text': 'It is now argued that ML, and more globally artificial intelligence (AI), will dramatically improve prognosis within the coming years.',
  'labels': [],
  'entities': []},
 {'text': 'Simultaneously, progress made in high-throughput technologies has contributed to developing new fields such as metagenomics.',
  'labels': [],
  'entities': []},
 {'text': 'The association of the gut microbiota with human health and disease has been widely discussed, and links with numerous diseases are described.',
  'labels': [],
  'entities': []},
 {'text': 'Specifically, ecological relationships among bacterial species such as mutualism, parasitism, and competi

In [37]:
for sents in sentencesSplited:
    print(sents.to_tagged_string())
    print("#"*30)
    for annotation_layer in sents.annotation_layers.keys():
        print(annotation_layer)
        for entity in sents.get_spans(annotation_layer):
            print("*"*30)
            print(entity)

  Consumption of alcohol <S-Chemical> ( ethanol <S-Chemical> ) in various forms has been an integral part of human <S-Species> civilization .
##############################
Since ages , it also has been an important cause of death and health impairment across the globe .
##############################
Ethanol <S-Chemical> - mediated liver <B-Disease> injury <E-Disease> , known as alcoholic <B-Disease> liver <I-Disease> disease <E-Disease> ( ALD <S-Disease> ) , is caused by surplus intake of alcohol <S-Chemical> .
##############################
Several studies have proposed the different pathways that may be lead to ALD <S-Disease> .
##############################
One of the factors that may affect the cytochrome <B-Gene> P450 <E-Gene> ( CYP2E1 <S-Gene> ) metabolic pathway is gut dysbiosis .
##############################
The gut microbiota produces various compounds that play an important role in regulating healthy functions of distal organs such as the adipose tissue and liver .
#####

In [33]:
sents.annotation_layers

{}

In [6]:
from bioFlairBasedNER import FlairBasedNER

In [7]:
fbNER=FlairBasedNER()

2022-08-02 18:22:35,576 loading file C:\Users\aberhe\.flair\models\hunflair-celline-v1.0.pt
2022-08-02 18:22:40,973 loading file C:\Users\aberhe\.flair\models\hunflair-chemical-full-v1.0.pt
2022-08-02 18:22:46,222 loading file C:\Users\aberhe\.flair\models\hunflair-disease-full-v1.0.pt
2022-08-02 18:22:53,234 loading file C:\Users\aberhe\.flair\models\hunflair-gene-full-v1.0.pt
2022-08-02 18:22:59,353 loading file C:\Users\aberhe\.flair\models\hunflair-species-full-v1.1.pt


In [8]:
senteces=fbNER.tagDocument(artDet.abstract.strip())

In [9]:
df=fbNER.extractNER(outputFile="NER_first_output",outputFormat="csv")

In [10]:
df

,sentence,entity,start_chunck,end_chunk,label,score
0,Consumption of alcohol (ethanol) in various fo...,alcohol,15,22,Chemical,0.974
1,Consumption of alcohol (ethanol) in various fo...,ethanol,24,31,Chemical,0.989
2,Consumption of alcohol (ethanol) in various fo...,human,79,84,Species,0.993
3,"Ethanol-mediated liver injury, known as alcoho...",Ethanol,0,7,Chemical,0.992
4,"Ethanol-mediated liver injury, known as alcoho...",alcohol,102,109,Chemical,0.951
5,"Ethanol-mediated liver injury, known as alcoho...",liver injury,17,29,Disease,0.732
6,"Ethanol-mediated liver injury, known as alcoho...",alcoholic liver disease,40,63,Disease,0.932
7,"Ethanol-mediated liver injury, known as alcoho...",ALD,65,68,Disease,1.000
8,Several studies have proposed the different pa...,ALD,73,76,Disease,0.999
9,One of the factors that may affect the cytochr...,cytochrome P450,39,54,Gene,0.761


In [13]:
jsonData=fbNER.extractNER(outputFile="NER_first_output",outputFormat="json")

In [14]:
jsonData

[{'setence': 'Consumption of alcohol (ethanol) in various forms has been an integral part of human civilization.',
  'entities': [{'text': 'alcohol',
    'start_chunk': 15,
    'end_chunk': 22,
    'label': 'Chemical',
    'score': 0.9738338589668274},
   {'text': 'ethanol',
    'start_chunk': 24,
    'end_chunk': 31,
    'label': 'Chemical',
    'score': 0.9892321825027466},
   {'text': 'human',
    'start_chunk': 79,
    'end_chunk': 84,
    'label': 'Species',
    'score': 0.9929606318473816}]},
 {'setence': 'Since ages, it also has been an important cause of death and health impairment across the globe.',
  'entities': []},
 {'setence': 'Ethanol-mediated liver injury, known as alcoholic liver disease (ALD), is caused by surplus intake of alcohol.',
  'entities': [{'text': 'Ethanol',
    'start_chunk': 0,
    'end_chunk': 7,
    'label': 'Chemical',
    'score': 0.9916825294494629},
   {'text': 'alcohol',
    'start_chunk': 102,
    'end_chunk': 109,
    'label': 'Chemical',
    'sc

In [116]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")

model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")

Downloading:   0%|          | 0.00/373 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/695k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

In [125]:
from transformers import pipeline
import numpy as np

In [119]:
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple") # pass device=0 if using gpu
pipe("""The patient reported no recurrence of palpitations at follow-up 6 months after the ablation.""")

[{'entity_group': 'Sign_symptom',
  'score': 0.9999311,
  'word': 'pal',
  'start': 38,
  'end': 41},
 {'entity_group': 'Sign_symptom',
  'score': 0.9063319,
  'word': '##pitations',
  'start': 41,
  'end': 50},
 {'entity_group': 'Clinical_event',
  'score': 0.99975544,
  'word': 'follow',
  'start': 54,
  'end': 60},
 {'entity_group': 'Date',
  'score': 0.999867,
  'word': '6 months after',
  'start': 64,
  'end': 78}]

In [121]:
artDet.abstract.strip()

'Microbiome biomarker discovery for patient diagnosis, prognosis, and risk evaluation is attracting broad interest. Selected groups of microbial features provide signatures that characterize host disease states such as cancer or cardio-metabolic diseases. Yet, the current predictive models stemming from machine learning still behave as black boxes and seldom generalize well. Their interpretation is challenging for physicians and biologists, which makes them difficult to trust and use routinely in the physician–patient decision-making process. Novel methods that provide interpretability and biological insight are needed. Here, we introduce “predomics”, an original machine learning approach inspired by microbial ecosystem interactions that is tailored for metagenomics data. It discovers accurate predictive signatures and provides unprecedented interpretability. The decision provided by the predictive model is based on a simple, yet powerful score computed by adding, subtracting, or divid

In [152]:
sents=artDet.fullArticle.strip()
sents=nltk.sent_tokenize(sents)

In [154]:
for sent in sents:
    pred=pipe(sent)
    if len(pred)>0:
        if len(pred)<2:
            print(pred[0]['entity_group'],pred[0]['score'],pred[0]['word'],len(pred))
            #i = np.argmax([x['score'] for x in pred])
        else:
            words=[]
            entGroup=[]
            for k in range(len(pred)):
                words+=[pred[k]['word']]
                entGroup+=[pred[k]['entity_group']]
            enti_joined=" ".join(words)
            entity_groupSe=list(set(entGroup))
            print(entity_groupSe,enti_joined,len(pred))

['Detailed_description'] high - throughput molecular and imaging technologies machine learning 2
Lab_value 0.97734386 improve 1
['Detailed_description', 'Lab_value'] progress high - throughput technologies meta 3
Diagnostic_procedure 0.5880131 ecological 1
['Diagnostic_procedure', 'Detailed_description', 'Lab_value'] metagenomics small number of samples number variables 5
['Diagnostic_procedure', 'Detailed_description'] micro catalogue millions 3
Coreference 0.8570305 models 1
['Diagnostic_procedure', 'Detailed_description'] learning algorithms dimension reduction regularization methods logistic regression elastic - net regularization ( en ##et support vector machine sv 11
Coreference 0.8491056 algorithms 1
Biological_structure 0.5493733 ml 1
Diagnostic_procedure 0.9861579 interpretable models 1
['Detailed_description', 'Coreference'] they operations limited 3
Detailed_description 0.46022114 causal 1
['Diagnostic_procedure', 'Sign_symptom', 'Lab_value'] models ecosystem relationships s

In [9]:
import pandas as pd
import os

In [2]:
dataframe=pd.read_csv("All_Vellonella_Articles.csv")

In [3]:
dataframe.shape

(1499, 5)

In [5]:
tagger = Classifier.load("hunflair")
def extractSave(file_xml,bacteria_name="Veillonella",tagger=tagger):
    
    #file_xml='/data/projects/santal/data/PMC_Articles/oa_comm/xml/all/PMC10000167.xml'#PMC10000014.xml'
    with open(file_xml, 'r') as f:
        data = f.read()
    articleDetails=ad.ArticleDetails(file_xml)
    try:
        articleDetails.getArticleContent()
    except:
        return "FILE ERROR"
    sent_id_d=file_xml.split("/")[-1].replace(".xml","")
    folder_path="/home/aberhe/Projects/SANTAL/ASLR/Results/BioNER/"+bacteria_name
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    else:
        os.path.exists(folder_path+"/"+sent_id_d+".csv")
    
    df=pd.DataFrame(columns=("sent_id","sentence","entity","start_chunck","end_chunk","label","score"))
    i=0
    #for key in tqdm(articleDetails.sections):
    sentences = nltk.sent_tokenize(articleDetails.articleContent[key])
    for sentence in sentences:
        sentence=Sentence(sentence)
        tagger.predict(sentence)
        sent_id=sent_id_d+"_"+str(i)
        for labels in sentence.get_labels():
            #print(la.value,la.data_point.text,la.data_point.start_position,la.data_point.end_position)
            taggedEntities=[sent_id,sentence.tokenized,labels.data_point.text,labels.data_point.start_position,labels.data_point.end_position,labels.value,round(labels.score,3)]
            df.loc[len(df)]=taggedEntities
        i=i+1
    folder_path="/home/aberhe/Projects/SANTAL/ASLR/Results/BioNER/"+bacteria_name
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
        
    df.to_csv(folder_path+"/"+sent_id_d+".csv")

#if __name__=="__main__":
with open("/home/aberhe/Projects/SANTAL/ASLR/data/Veillonella_articles_found_exits77.txt", "r") as f:
    files_veillonella=f.readlines()
for file in tqdm(files_veillonella):
    bacteria_name="Veillonella"
    folder_path="/home/aberhe/Projects/SANTAL/ASLR/Results/BioNER/"+bacteria_name
    sent_id_d=file.split("/")[-1].replace(".xml","")
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
        try:
            extractSave(file_xml=file.replace("\n",""),bacteria_name=bacteria_name)
        except:
            continue
    else:
        if os.path.exists(folder_path+"/"+sent_id_d+".csv"):
            continue
        else:
            try:
                extractSave(file_xml=file.replace("\n",""),bacteria_name=bacteria_name)
            except:
                continue

2023-06-29 13:27:23,596 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, B-Disease, E-Disease, I-Disease, S-Disease, <START>, <STOP>
2023-06-29 13:27:24,336 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, S-Gene, B-Gene, I-Gene, E-Gene, <START>, <STOP>
2023-06-29 13:27:25,045 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, S-Species, B-Species, I-Species, E-Species, <START>, <STOP>
2023-06-29 13:27:25,784 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, S-CellLine, B-CellLine, I-CellLine, E-CellLine, <START>, <STOP>
2023-06-29 13:27:26,566 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, S-Chemical, B-Chemical, I-Chemical, E-Chemical, <START>, <STOP>


100%|██████████| 7872/7872 [00:02<00:00, 2921.83it/s]


In [179]:
len(files_veillonella)

7872

In [174]:
len(files_veillonella)

0

In [175]:
files_veillonella

[]

In [176]:
with open("/home/aberhe/Projects/SANTAL/ASLR/data/Veillonella_articles_found_exits.txt", "r") as f:
    files_veillonella=f.readlines()